# Introduction to Dask
by Dr Stef Garasto

<p>
Dask is a popular python library for parallel computing. It is designed to integrate easily with existing python code and other common libraries like Numpy and Pandas. During this introductory tutorial you will get to know some fundamental Dask functions, like dask.delayed and dask.dataframe, and workflows. There will also be some hands-on coding exercises for you to practice your newfound Dask knowledge. 
</p>

### References (and sources of inspiration):
[1] Dask Tutorial by https://github.com/dask/dask-tutorial/

[2] Dask documentation https://docs.dask.org/en/latest/

Technical reminder: press shift-enter to execute single cells in this notebook.

# Setup

In [ ]:
# Setup
LOCAL = True
if not LOCAL:
    # Install some requirements
    !pip install snakeviz
    %pip install "tornado>=5" 
    !pip -q install dask
    !pip -q install distributed
    #!pip -q install --upgrade --ignore-installed numpy pandas scipy sklearn
    !pip -q install graphviz 
    !apt-get install graphviz -qq
    !pip -q install pydot
    !pip -q install bokeh


In [ ]:
if not LOCAL:
    # to get the data
    !git clone https://github.com/stefgrs/dask-mini-tutorial

In [ ]:
# setup the Dask scheduler (just go with it for now!)
from dask.distributed import Client, progress

client = Client(n_workers=4, processes = False)
client



# Dask delayed


dask.delayed is a Dask interface that allows users to parallelise custom algorithms with a light annotation of normal python code.

In [ ]:
from dask import delayed

In [ ]:
from time import sleep

# define some auxiliary functions - we simulate computational time with the 'sleep' function
def inc(x):
    sleep(1)
    return x + 1

def add(x, y):
    sleep(1)
    return x + y


Note: the command '%%time' will measure the time execution of all the python statements in a cell. It prints two values: the CPU time and the Wall time. CPU time is the time that the CPU was busy. Wall time is the overall time that it took for the code to execute (it includes the time the CPU was busy and the time spent waiting).

In [ ]:
%%time
# This takes three seconds to run because we call each
# function sequentially, one after the other

x = inc(1)
y = inc(2)
z = add(x, y)

print(f'The final sum is {z}')


In [ ]:
%%time
# When we use 'delayed' we are telling Dask to perform these computations in a 
# lazy way. Unless explicitely asked, Dask will only build a description of the 
# computations needed to obtain the desired result, but it will not do the 
# computation itself - not yet at least! 
#  

del_x = delayed(inc)(1)
del_y = delayed(inc)(2)
del_z = delayed(add)(del_x, del_y)


It didn't take long now, did it?
But wait...

In [ ]:
# What is the difference from before? What do all the variables look like?
print(f"Without applying 'delayed' the variables x, y, z are {x}, {y}, {z}")
print(f"Applying 'delayed' the delayed variables x, y, z are {del_x}, {del_y}, {del_z}")


In [ ]:
# What does the computation graph look like?

del_z.visualize()


In [ ]:
%%time
# What happens when we tell Dask to execute the computation?
z_exec = del_z.compute()
print(f'The final sum is {z_exec}')


In [ ]:
'''
We can use dask.delayed in two different ways:
- as a wrapper around a function, as we saw in the cell above, i.e. delayed(inc)
- as a decorator associated with a function, as you can see below in this cell.

When using it as a wrapper, only the instances of a function that are wrapped 
with delayed will actually be delayed.
When using it as a decorator, all the instances will be delayed.

@delayed
def inc(x):
    return x + 1

@delayed
def add(x, y):
    return x + y
'''
print()


If you want to see how Dask visualises what's happening in the background of this computation have a look at [this video](https://www.youtube.com/watch?v=SHqFmynRxVU&feature=youtu.be).

## Dask delayed hands-on

Over to you with a slightly more complicated example.

Background:

Let's assume we run an experiment where we split participants in two groups (one with 60% of participants, one with 40%), gave them a test and collected the average score for each group separately.
We saved these scores in two separate files, and the sizes of the two groups (as proportions, that is 0.6 and 0.4) in two other files. We want to load the numbers from files and compute the weighted average of the
final group scores.


In [ ]:
%%time
# Translated into standard python, we can get the weighted average like this:

import os

data_folder = 'data/'
scores_file_names = os.path.join(data_folder, 'score_group{x}.txt')
weights_file_names = os.path.join(data_folder,'weight_group{x}.txt')

# defining custom function 
# again we will pretend they are more computationally expensive than they actually are
def load_value_from_text(file_name):
    with open(file_name, 'r') as f:
        data = f.read()
    sleep(1)
    return float(data)

def mul(x,y):
    sleep(1)
    return x*y

def add_list(input_list):
    sleep(1)
    return sum(input_list)

# load numbers from files
survey_scores = [load_value_from_text(scores_file_names.format(x=i+1)) for i in range(2)]
survey_weights = [load_value_from_text(weights_file_names.format(x=i+1)) for i in range(2)]

# multiply the average score for each group by the group proportion wrt the total
# number of participants
intermediate_scores = []
for score, weight in zip(survey_scores, survey_weights):
    intermediate_scores.append(mul(score,weight))

# sum the weighted scores to obtain the weighted average
total = add_list(intermediate_scores)

# show the result:
total


### Exercise 1: apply delayed to the computation

Will give you a few minutes to write your code, then I will show the solution.

In [ ]:
%%time
# load numbers from files
######################
### Your code here ###
######################

# multiply the average score for each group by the group proportion wrt the total
# number of participants
intermediate_scores = []
for score, weight in zip(survey_scores, survey_weights):
######################
### Your code here ###
######################

# sum the weighted scores to obtain the weighted average
######################
### Your code here ###
######################


### Excercise 2: what does the computation graph look like?
Is it graph A or graph B? Post your answers in the chat.

![picture](https://drive.google.com/uc?export=view&id=1IAhtcSphBaLUIfViTknBMjMtIBk-LHXG)

In [ ]:
# let's see what it looks like
######################
### Your code here ###
######################


### Excercise 3: how long do you think Dask will take to compute the results?
Post your answers in the chat

In [ ]:
%%time
# let's see how long it'll take
######################
### Your code here ###
######################


## Dask delayed notes


1. Operations that are supported on delayed objects include the following:


*   If x is a delayed result, then performing arithmetic operations on it also produces a delayed result.
*   Slicing a delayed object also produces a delayed object.
*   If x is a delayed object with methods and attributes we can access, those will also be delayed objects.

2.   Operations which are not supported include iteration (for) and bool (predicate).

3.   Calling delayed_obj.compute() works well for single outputs, but for multiple outputs it might be best to use the dask.compute function.

So, for example...


In [ ]:
import numpy as np
# let's get our old friends back...
del_x = delayed(inc)(1)
del_y = delayed(inc)(2)
del_z = delayed(add)(del_x, del_y)

# ... and some new ones
del_range = delayed(np.arange)(10)


In [ ]:
# let's check what's still a delayed object
# arithmetic operations
print(type(del_z+1))
print(type(del_z*10))

# method calls
print(type(del_range.shape))

# slicing
print(type(del_range[::2]))


In [ ]:
from dask import compute

def sleeping_square(x):
    sleep(1)
    return x ** 2

square_range = delayed(sleeping_square)(np.arange(10))
square_range


In [ ]:
%%time
#calling compute this way means that Dask can make good use of shared intermediate values
del_min, del_max = compute(square_range.min(), square_range.max()) 

print(f'Min: {del_min}, Max: {del_max}')


In [ ]:
%%time
#compare with this:
del_min2, del_max2 = (square_range.min().compute(), square_range.max().compute())


What <b>can't</b> we apply dask.delayed to?

In order to build the computation graph, dask needs to know what function it has to call.

When the choice of the function to use depend on a boolean condition (True or False), then Dask only knows which function to use if it knows the outcome of the boolean condition. If this is the case, do you think we can call delayed on the boolean function as well?

Let's see...

In [ ]:
%%time

# Let's consider this conditional for loop

def is_even(x):
    return x % 2

intermediate_res = []
for i in range(10):
    if is_even(i):
        y = inc(i)
    else:
        y = sleeping_square(i)
    intermediate_res.append(y)

total = sum(intermediate_res)


In [ ]:
# First, let's parallelise it using dask.delayed
# try it out for yourself!
%%time
intermediate_res = []
for i in range(6):
######################
### Your code here ###
######################


In [ ]:
# what does the graph look like?
total.visualize()


In [ ]:
# how long does the computation take?
%%time
total.compute()


In [ ]:
# What would have happened if we had tried to call delayed on the if condition?
intermediate_res = []
for i in range(6):
    if delayed(is_even)(i):
        y = delayed(inc)(i)
    else:
        y = delayed(sleeping_square)(i)
    intermediate_res.append(y)

#total = delayed(sum)(intermediate_res)
total = delayed(sum)(intermediate_res)


## The speed-up of dask delayed is hardware dependent

In [ ]:
def long_sum(n = 10):
    intermediate_res = []
    for i in range(n):
        if is_even(i):
            y = delayed(inc)(i)
        else:
            y = delayed(sleeping_square)(i)
        intermediate_res.append(y)

    total = delayed(sum)(intermediate_res)
    return total

from time import time as tt

x = range(4,24,2)

elapsed_times = []
for n in x: #range(4,24,2):
    t0 = tt()
    total = long_sum(n = n)
    _ = total.compute()
    elapsed_times.append(tt()-t0)
    
print('done')

In [ ]:
import matplotlib.pyplot as plt

plt.plot(x, elapsed_times)


What's happening in your graph? Post your comments in the chat.

## Dask delayed best practices

![picture](https://drive.google.com/uc?export=view&id=1UHrvS18W6Kpmln17dvRMPtwZJNfJCD71)


## Questions?